# Visualization for Core-level spectra

Welcome! Here you can plot theoretical core-level spectra from the data in the database.
This GUI just wraps some functionalities of the method 'plot_methods.plot_fleur_aiida.plot_spectra'.
You can export the data or save the plot. Enjoy! Created @ PGI-1 by Jens Broeder 

In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.data.folder import FolderData
from aiida.orm import load_node 

import ipywidgets as ipw
from IPython.display import display, clear_output
import nglview

import urlparse
from collections import OrderedDict
from pprint import pformat
import ase.io
from base64 import b64encode

from plot_methods.plot_fleur_aiida import plot_spectra
import plot_methods

In [ ]:
#?plot_spectra

In [ ]:
def prepare_input():
    pass
    inputs = {}
    inputs['energy_range'] = [0, 200]
    #node = load_node(int(inp_pk.value))
    pklist = inp_pks.value.strip().split()
    if pklist:
        pklist_new = []
        for pk in pklist:
            #try:
            pk_new = int(pk)
            #except:
            pklist_new.append(pk_new) 
        inputs['wc_nodes'] = pklist_new
    
    if inp_egrid.value:
        inputs['energy_grid'] = inp_egrid.value
    
    if inp_erange_min.value:
        inputs['energy_range'] = [inp_erange_min.value, inputs['energy_range'][1]]

    if inp_erange_max.value:
        inputs['energy_range'] = [inputs['energy_range'][0], inp_erange_max.value]

    if inp_fwhm_l.value:
        inputs['fwhm_l'] = inp_fwhm_l.value

    if inp_fwhm_g.value:
        inputs['fwhm_g'] = inp_fwhm_g.value

    peakfunc = inp_peakfun.value.strip().split()
    if peakfunc:        
        if len(peakfunction) != 1:
            print('You can only provide one peakfunction name. I use voigts as default...')
        else:
            inputs['peakfunction'] = peakfunc[0]
    
    factors = inp_factors.value.strip().split()
    if factors:        
        if len(factors) != len(pklist):
            print('The number of factors you provide must be equals to the number of nodes provided. I skip these factors...')
        else:
            factors_new = []
            for factor in factors:
                factors_new.append(float(factor))
            inputs['factors'] = factors_new
            print(factors_new)
        
    return inputs

#plot_spectra(wc_nodes, title='', factors=[], energy_range=[100, 120], fwhm_g=0.6, fwhm_l=0.1, energy_grid=0.2, peakfunction='voigt', linetyp_spec='o-', warn_ref=False, **kwargs)

In [ ]:
# search UI
style = {"description_width":"100px"}
layout = ipw.Layout(width="692px")
btn_sub = ipw.Button(description="Plot")
inp_pks = ipw.Text(description="Nodes PK/UUID: ", placeholder='e.g. 4062 4753 (space separated)', layout=layout, style=style)
inp_factors = ipw.Text(description="Factors:", placeholder='e.g. 5.1 1 1 (space seperator # of factors has to be equal # of pks)', layout=layout, style=style)
inp_erange_min = ipw.BoundedFloatText(value=0, min=0, max=2000, step=0.1, description='Emin [eV]:', disabled=False, 
                            continuous_update=False, readout_format='.1f')
inp_erange_max = ipw.BoundedFloatText(value=200, min=0, max=2000, step=0.1, description='Emax [eV]:', disabled=False, 
                            continuous_update=False,  readout_format='.1f')
inp_egrid = ipw.FloatSlider(value=0.002, min=0, max=0.5, step=0.001, description='Energy grid [eV]:', disabled=False, 
                            continuous_update=False, orientation='horizontal', readout=True, readout_format='.3f')
inp_fwhm_l = ipw.FloatSlider(value=0.1, min=0, max=2.0, step=0.01, description='fwhm_l [eV]:', disabled=False, 
                            continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
inp_fwhm_g = ipw.FloatSlider(value=0.6, min=0, max=2.0, step=0.01, description='fwhm_g [eV]:', disabled=False, 
                            continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
inp_peakfun = ipw.Text(description="Peakfunction type:", placeholder='e.g. voigt gaus lorentz pseudo-voigt (only one)', layout=layout, style=style)




function_user_input = [inp_pks, inp_factors, inp_erange_min, inp_erange_max, inp_egrid, inp_fwhm_l, inp_fwhm_g, inp_peakfun]

In [ ]:
def on_submit(b):
    global stack_list
    with output:
        clear_output()
        function_input = prepare_input()
        data = plot_spectra(**function_input)
        #print(data)

inp_pks.on_submit(on_submit)
btn_sub.on_click(on_submit)
output = ipw.Output()
app = ipw.VBox(children=function_user_input + [btn_sub, output])
display(app)

In [ ]:
#stack_list = ipw.VBox()
#display(stack_list)

In [ ]:
# parse URL
url = urlparse.urlsplit(jupyter_notebook_url)
querypart = urlparse.parse_qs(url.query)
if 'pk' in querypart.keys():
    #print(querypart['pk'])
    pksting = ''
    for pk in querypart['pk']:
        pksting = pksting + '{} '.format(pk)
    inp_pks.value = pksting.strip()
    on_submit(None)